<a href="https://colab.research.google.com/github/peperjet/tutorial/blob/main/today_260129.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
# 1. Kaggle 데이터 준비하기

import pandas as pd

#파일 경로에 맞게 수정하기, 오류 발생하는 줄은 과감히 건너띄기
# ParserError: Error tokenizing data. C error: EOF inside string starting at row 25765 오류가 발생했습니다.
# 이는 파일의 25765번째 줄 근처에서 큰따옴표로 시작된 문자열이 닫히지 않은 채 파일이 끝났을 때 발생합니다.
# 해결책으로, 파이썬 엔진을 사용하면 더 유연하게 처리할 수 있습니다. (C 엔진보다 느릴 수 있습니다.)

df = pd.read_csv('/content/sample_data/winemag_data.csv',
                 index_col=0,
                 on_bad_lines='skip',
                 engine='python')  # engine='python'추가해서 더 똑똑하게 읽게 하기

# 데이터 로드 확인!
print(f"로드된 데이터 행 개수 : {len(df)}")
print(df.head())





로드된 데이터 행 개수 : 25762
    country                                        description  \
0     Italy  Aromas include tropical fruit, broom, brimston...   
1  Portugal  This is ripe and fruity, a wine that is smooth...   
2        US  Tart and snappy, the flavors of lime flesh and...   
3        US  Pineapple rind, lemon pith and orange blossom ...   
4        US  Much like the regular bottling from 2012, this...   

                          designation  points  price           province  \
0                        Vulkà Bianco      87    NaN  Sicily & Sardinia   
1                            Avidagos      87   15.0              Douro   
2                                 NaN      87   14.0             Oregon   
3                Reserve Late Harvest      87   13.0           Michigan   
4  Vintner's Reserve Wild Child Block      87   65.0             Oregon   

              region_1           region_2         taster_name  \
0                 Etna                NaN       Kerin O’Keefe   
1

In [23]:
# [insert 메소드] 1번 위치(country 열 옆)에 'Grade'라는 열을 만들고 'A'라고 채우기

df. insert(1, 'Grade', 'A')
print("--- insert 후 결과 (상위  3개) ---")
print(df[['country', 'Grade']].head(3))


# [pop 메소드] 방금 만든 'Grade' 열을 쏙 뽑아내서 삭제하기
grade_series = df.pop('Grade')
print("\n--- pop 후 결과 (Grade 열이 사라졌는지 확인) ---")
print(df.columns)


# insert & pop 열 추가와 삭제

# 오류 방지를 위해 'rating_10' 열이 이미 존재한다면 먼저 삭제합니다.
if 'rating_10' in df.columns:
    df.pop('rating_10')

# [insert] 4번 인덱스에 'rating_10' 열 추가 (points를 10의로 나눈 값)
df.insert(4, 'rating_10', df['points'] / 10)
print("1. insert 후 결과:\n", df[['points', 'rating_10']].head(3))


# [pop] 추가했던 'rating_10' 열을 삭제하며 변수에 저장
popped_col = df.pop('rating_10')
print("\n2. pop 완료: 'rating_10' 열이 삭제되었습니다.")

--- insert 후 결과 (상위  3개) ---
    country Grade
0     Italy     A
1  Portugal     A
2        US     A

--- pop 후 결과 (Grade 열이 사라졌는지 확인) ---
Index(['country', 'description', 'designation', 'points', 'price', 'province',
       'region_1', 'region_2', 'taster_name', 'taster_twitter_handle', 'title',
       'variety', 'winery'],
      dtype='object')
1. insert 후 결과:
    points  rating_10
0      87        8.7
1      87        8.7
2      87        8.7

2. pop 완료: 'rating_10' 열이 삭제되었습니다.


In [29]:
# copy & drop / truncate 메소드 구현

# [copy 메소드] 원본 데이터를 안전하게 복사
df_wine = df.copy ()

# [drop 메소드] 'description'과 'taster_twitter_handle' 두 개의 열을 삭제하기
# axis=1은 열을 의미합니다

# KeyError는 'description'과 'taster_twitter_handle' 열이 df_wine DataFrame에 존재하지 않을 때 발생합니다.
# 오류를 무시하고 진행하려면 errors='ignore' 옵션을 사용합니다.
# 또한, 'df=wine'은 문법 오류이므로 'wine'으로 수정합니다.
wine = df_wine.drop(['description', 'taster_twitter_handle'], axis=1, errors='ignore')

# [truncate 메소드] 데이터가 너무 많으니 0번부터 50번 인덱스까지만 자르기
# truncate 메소드는 인덱스가 정렬되어 있어야 합니다.
df_wine = df_wine.sort_index()
df_small = df_wine.truncate(before=0, after=50)


print("--- drop & truncate 결과---")
print(f"남은 컬럼들: {df_small.columns}")
print(f"남은 데이터 개수 : {len(df_small)}")
df_small.head()

--- drop & truncate 결과---
남은 컬럼들: Index(['country', 'designation', 'points', 'price', 'province', 'region_1',
       'region_2', 'taster_name', 'title', 'variety', 'winery'],
      dtype='object')
남은 데이터 개수 : 51


,country,designation,points,price,province,region_1,region_2,taster_name,title,variety,winery
0,Italy,Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,Portugal,Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,US,NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
3,US,Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
4,US,Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks


In [32]:
# [concat 메소드]

# 1. df_small의 위쪽 3개와 아래쪽 3개를 준비합니다.
top = df_small.head(3)
bottom = df_small.tail(3)

# 2. 리스트에 담아서 합칩니다.
# 공식 문서 스타일대로 합쳐진 결과에 'Top'과 'Bottom'이라는 라벨 붙이기
df_concat = pd.concat([top, bottom], keys=['Top', 'Bottom'])

print("--- concat 완료 ---")
print(df_concat)

--- concat 완료 ---
            country   designation  points  price           province  \
Top    0      Italy  Vulkà Bianco      87    NaN  Sicily & Sardinia   
       1   Portugal      Avidagos      87   15.0              Douro   
       2         US           NaN      87   14.0             Oregon   
Bottom 48        US           NaN      86   16.0           Virginia   
       49    France    Eté Indien      86   14.0         Beaujolais   
       50     Italy        Scialo      86    NaN  Sicily & Sardinia   

                    region_1           region_2    taster_name  \
Top    0                Etna                NaN  Kerin O’Keefe   
       1                 NaN                NaN     Roger Voss   
       2   Willamette Valley  Willamette Valley   Paul Gregutt   
Bottom 48         Monticello                NaN            NaN   
       49           Brouilly                NaN     Roger Voss   
       50            Sicilia                NaN            NaN   

                     

In [33]:
# 중복행 제거 (drop_duplicates) 구현

# [drop_duplicates] 메소드
# 'country'가 같은 데이터 중복이라고 가정하게 제거하기
# 실제로는 'title'을 기준으로 많이함. 오늘은 'country'로

# keep='first' 중복 중 첫 번째 것만 남기겠다
df_final = df_concat.drop_duplicates(subset=['country'], keep='first')

print("--- drop_duplicates 결과 ---")
print(f"중복 제거 전 행 개수 : {len(df_concat)}")
print(f"중복 제거 후 행  개수 : {len(df_final)}")
df_final

--- drop_duplicates 결과 ---
중복 제거 전 행 개수 : 6
중복 제거 후 행  개수 : 4


country   designation  points  price           province  \
Top    0      Italy  Vulkà Bianco      87    NaN  Sicily & Sardinia   
       1   Portugal      Avidagos      87   15.0              Douro   
       2         US           NaN      87   14.0             Oregon   
Bottom 49    France    Eté Indien      86   14.0         Beaujolais   

                    region_1           region_2    taster_name  \
Top    0                Etna                NaN  Kerin O’Keefe   
       1                 NaN                NaN     Roger Voss   
       2   Willamette Valley  Willamette Valley   Paul Gregutt   
Bottom 49           Brouilly                NaN     Roger Voss   

                                                      title         variety  \
Top    0                  Nicosia 2013 Vulkà Bianco  (Etna)     White Blend   
       1      Quinta dos Avidagos 2011 Avidagos Red (Douro)  Portuguese Red   
       2      Rainstorm 2013 Pinot Gris (Willamette Valley)      Pinot Gris   
Bottom 49  Vignerons de Bel Air 2011 Eté Indien  (Brouilly)           Gamay   

                         winery  
Top    0                Nicosia  
       1    Quinta dos Avidagos  
       2              Rainstorm  
Bottom 49  Vignerons de Bel Air